# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("logregconsult").getOrCreate()

In [3]:
df = spark.read.csv("customer_churn.csv", inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [6]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [9]:
output = assembler.transform(df)

In [10]:
final_data = output.select('features', 'churn')

In [11]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [12]:
from pyspark.ml.classification import LogisticRegression

In [13]:
lr = LogisticRegression(featuresCol='features', labelCol='churn')

In [14]:
fitted_churn_model = lr.fit(train_churn)

In [15]:
training_sum = fitted_churn_model.summary

In [16]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                629|                629|
|   mean|0.17011128775834658|0.13354531001589826|
| stddev|0.37602956799028064|0.34043386040459533|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol="churn")

In [20]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [21]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[27.0,8628.8,1.0,...|    0|[5.41763821568662...|[0.99558199045486...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.09223046971143...|[0.99969418687023...|       0.0|
|[28.0,11245.38,0....|    0|[3.86512008837547...|[0.97946991374155...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.63974813021398...|[0.99043229486327...|       0.0|
|[29.0,12711.15,0....|    0|[5.57478345524188...|[0.99622203048609...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.97881928937623...|[0.99747458002563...|       0.0|
|[30.0,8677.28,1.0...|    0|[3.95068273169895...|[0.98112168772059...|       0.0|
|[30.0,10183.98,1....|    0|[2.87131425353242...|[0.94641004351988...|       0.0|
|[30.0,12788.37,0....|    0|[2.81057529513332...|[0.94324462501184...|       0.0|
|[31.0,5387.75,0

In [23]:
AUC = eval.evaluate(pred_and_labels.predictions)

In [24]:
AUC

0.7338841289269686

In [25]:
spark.stop()